This notebook calculates the Bayes factor for the Cubic-Spline interpolation against the null model, Smooth Broken Power-Law function, used by L22 in single-burst Bayesian analysis.

In [2]:
import pandas as pd
import numpy as np

z_n1_liu = "../Data/posteriors_SBPL_L22/ln_evidence_subL_n1.txt"
z_n1_du = '../Data/posteriors_cubicspline_DU25/ln_evidence_subL_n1.txt'
z_n2_liu = "../Data/posteriors_SBPL_L22/ln_evidence_subL_n2.txt"
z_n2_du = '../Data/posteriors_cubicspline_DU25/ln_evidence_subL_n2.txt'
out_file = "../Data/lnBayesFactor.txt"

df_n1_liu = pd.read_csv(z_n1_liu, sep="\t", index_col=False)
df_n2_liu = pd.read_csv(z_n2_liu, sep="\t", index_col=False)
df_n1_du = pd.read_csv(z_n1_du, sep=",", skiprows=0)
df_n2_du = pd.read_csv(z_n2_du, sep=",", skiprows=0)

df_n1_liu.columns = df_n1_liu.columns.str.strip()
df_n2_liu.columns = df_n2_liu.columns.str.strip()
df_n1_du.columns = df_n1_du.columns.str.strip()
df_n2_du.columns = df_n2_du.columns.str.strip()

df_n1_liu['Name'] = df_n1_liu['Name'].astype(str).str.strip()
df_n2_liu['Name'] = df_n2_liu['Name'].astype(str).str.strip()
df_n1_du['Name'] = df_n1_du['Name'].astype(str).str.strip()
df_n2_du['Name'] = df_n2_du['Name'].astype(str).str.strip()

df_merged_n1 = pd.merge(df_n1_liu, df_n1_du, on="Name", suffixes=("_liu", "_du"))
df_merged_n2 = pd.merge(df_n2_liu, df_n2_du, on="Name", suffixes=("_liu", "_du"))

df_result = df_merged_n1[["Name"]].copy()

df_result["lnBF_n1"] = df_merged_n1["logE-evidence_du"] - df_merged_n1["logE-evidence_liu"]
df_result["lnBF_n1_err"] = np.sqrt(df_merged_n1["error_du"]**2 + df_merged_n1["error_liu"]**2)
df_result["lnBF_n2"] = df_merged_n2["logE-evidence_du"] - df_merged_n2["logE-evidence_liu"]
df_result["lnBF_n2_err"] = np.sqrt(df_merged_n2["error_du"]**2 + df_merged_n2["error_liu"]**2)

df_result.to_csv(out_file, sep="\t", index=False)

print(f"lnBayesFactor file saved as: {out_file}")
print(df_result.head())

lnBayesFactor file saved as: ../Data/lnBayesFactor.txt
         Name    lnBF_n1  lnBF_n1_err    lnBF_n2  lnBF_n2_err
0  GRB080916C   3.692106     0.127532   3.254751     0.117721
1   GRB081221  11.726467     0.124750  11.807080     0.117620
2   GRB090328  15.721682     0.108257  16.085650     0.105194
3   GRB090618   2.667774     0.143249   2.996302     0.139632
4  GRB090926A  16.001352     0.155707  13.766698     0.139196


In [ ]:
# create latex table
import pandas as pd

df = pd.read_csv('lnBayesFactor.txt', sep='\t')

def fmt(val, err):
    return f"${val:.2f} \\pm {err:.2f}$"

df['lnBF_n1_fmt'] = df.apply(lambda row: fmt(row['lnBF_n1'], row['lnBF_n1_err']), axis=1)
df['lnBF_n2_fmt'] = df.apply(lambda row: fmt(row['lnBF_n2'], row['lnBF_n2_err']), axis=1)

latex_lines = []
latex_lines.append("\\begin{table}[htbp]")
latex_lines.append("\\centering")
latex_lines.append("\\caption{Bayes factors for GRBs (with uncertainties).}")
latex_lines.append("\\begin{tabular}{lcc}")
latex_lines.append("\\hline")
latex_lines.append("Name & $\\ln BF_{n1}$ & $\\ln BF_{n2}$ \\\\")
latex_lines.append("\\hline")

for _, row in df.iterrows():
    line = f"{row['Name']} & {row['lnBF_n1_fmt']} & {row['lnBF_n2_fmt']} \\\\"
    latex_lines.append(line)

latex_lines.append("\\hline")
latex_lines.append("\\end{tabular}")
latex_lines.append("\\label{tab:grb_lnBF}")
latex_lines.append("\\end{table}")

with open('lnBF_table_tex.txt', 'w') as f:
    for line in latex_lines:
        f.write(line + '\n')
